## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lensk,RU,60.7253,114.9278,-35.10,80,100,2.68,overcast clouds
1,1,Mar Del Plata,AR,-38.0023,-57.5575,64.40,55,90,23.02,overcast clouds
2,2,Avarua,CK,-21.2078,-159.7750,75.20,94,98,1.14,overcast clouds
3,3,Kokoda,PG,-8.8785,147.7407,77.05,65,100,6.44,moderate rain
4,4,Hasaki,JP,35.7333,140.8333,26.60,80,20,14.41,few clouds
5,5,Castro,BR,-24.7911,-50.0119,72.63,86,90,5.97,overcast clouds
6,6,Miguel Hidalgo,MX,19.4225,-99.2028,66.20,39,40,3.44,scattered clouds
7,7,Volsk,RU,52.0333,47.3833,33.69,96,100,10.13,overcast clouds
8,8,Soyo,AO,-6.1349,12.3689,81.50,80,98,9.73,overcast clouds
9,9,Rikitea,PF,-23.1203,-134.9692,79.07,74,74,10.80,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Avarua,CK,-21.2078,-159.7750,75.20,94,98,1.14,overcast clouds
3,3,Kokoda,PG,-8.8785,147.7407,77.05,65,100,6.44,moderate rain
8,8,Soyo,AO,-6.1349,12.3689,81.50,80,98,9.73,overcast clouds
9,9,Rikitea,PF,-23.1203,-134.9692,79.07,74,74,10.80,light rain
10,10,Jacareacanga,BR,-6.2222,-57.7528,76.73,87,100,3.04,moderate rain
11,11,Cape Town,ZA,-33.9258,18.4232,78.80,47,20,11.50,few clouds
14,14,Omboue,GA,-1.5746,9.2618,79.77,81,55,8.05,moderate rain
17,17,Mitu,CO,1.1983,-70.1733,81.77,88,77,3.11,broken clouds
19,19,Pandan,PH,14.0453,124.1694,75.87,81,100,18.21,overcast clouds
24,24,Isla Mujeres,MX,21.2311,-86.7310,78.80,53,40,16.11,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             222
City                   222
Country                220
Lat                    222
Lng                    222
Max Temp               222
Humidity               222
Cloudiness             222
Wind Speed             222
Current Description    222
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.count()

Unnamed: 0             220
City                   220
Country                220
Lat                    220
Lng                    220
Max Temp               220
Humidity               220
Cloudiness             220
Wind Speed             220
Current Description    220
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Avarua,CK,75.20,overcast clouds,-21.2078,-159.7750,
3,Kokoda,PG,77.05,moderate rain,-8.8785,147.7407,
8,Soyo,AO,81.50,overcast clouds,-6.1349,12.3689,
9,Rikitea,PF,79.07,light rain,-23.1203,-134.9692,
10,Jacareacanga,BR,76.73,moderate rain,-6.2222,-57.7528,
11,Cape Town,ZA,78.80,few clouds,-33.9258,18.4232,
14,Omboue,GA,79.77,moderate rain,-1.5746,9.2618,
17,Mitu,CO,81.77,broken clouds,1.1983,-70.1733,
19,Pandan,PH,75.87,overcast clouds,14.0453,124.1694,
24,Isla Mujeres,MX,78.80,scattered clouds,21.2311,-86.7310,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Avarua,CK,75.20,overcast clouds,-21.2078,-159.7750,Paradise Inn
3,Kokoda,PG,77.05,moderate rain,-8.8785,147.7407,Daito Guest House
8,Soyo,AO,81.50,overcast clouds,-6.1349,12.3689,Hotel Mpampa
9,Rikitea,PF,79.07,light rain,-23.1203,-134.9692,Pension Maro'i
10,Jacareacanga,BR,76.73,moderate rain,-6.2222,-57.7528,Casa do Ian😎
11,Cape Town,ZA,78.80,few clouds,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
14,Omboue,GA,79.77,moderate rain,-1.5746,9.2618,Hotel Olako
17,Mitu,CO,81.77,broken clouds,1.1983,-70.1733,
19,Pandan,PH,75.87,overcast clouds,14.0453,124.1694,Bonifacio Lodge
24,Isla Mujeres,MX,78.80,scattered clouds,21.2311,-86.7310,Hotel La Joya Isla Mujeres


In [9]:
# 7. Drop the rows where there is no Hotel Name.
filtered_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
filtered_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Avarua,CK,75.20,overcast clouds,-21.2078,-159.7750,Paradise Inn
3,Kokoda,PG,77.05,moderate rain,-8.8785,147.7407,Daito Guest House
8,Soyo,AO,81.50,overcast clouds,-6.1349,12.3689,Hotel Mpampa
9,Rikitea,PF,79.07,light rain,-23.1203,-134.9692,Pension Maro'i
10,Jacareacanga,BR,76.73,moderate rain,-6.2222,-57.7528,Casa do Ian😎
11,Cape Town,ZA,78.80,few clouds,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
14,Omboue,GA,79.77,moderate rain,-1.5746,9.2618,Hotel Olako
19,Pandan,PH,75.87,overcast clouds,14.0453,124.1694,Bonifacio Lodge
24,Isla Mujeres,MX,78.80,scattered clouds,21.2311,-86.7310,Hotel La Joya Isla Mujeres
26,Bintulu,MY,78.80,few clouds,3.1667,113.0333,900 Boutique INN


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
filtered_hotel_df.to_csv(output_data_file)

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in filtered_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = filtered_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = filtered_hotel_df[["Lat", "Lng"]]
max_temp = filtered_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))